# The Knapsack Problem DQM

CDL Quantum Hackathon 2021  
Team ZebraKet   
Ziwei Qiu (ziweiqiu@g.harvard.edu), Alex Khan, Theo Cleland, Ehsan Torabizadeh

# Problem Definition

As a grocery store manager, you want to re-stock. This notebook helps you make decisions what items to buy to maximize your profit given an integer budget contrainst $W$. There are $M$ items in the market. $W_{\alpha}$ and $V_{\alpha}$ denotes the cost and profit of item $\alpha$, respectively. In the real world, there is an upper limit of the quantity we can buy for each item. We solved the optimization problem assuming two different bound conditions: (1) bound is fixed for all items, and (2) bound differs among items.

$$\text{maximize} \quad \sum_{\alpha=1}^{M} V_{\alpha} x_{\alpha}$$
$$\text{subject to} \quad \sum_{\alpha=1}^{M}W_{\alpha}x_{\alpha}\leq W$$
$$ 0 \leq x_{\alpha} \leq \text{bound}[\alpha], \quad x_{\alpha} \text{ is integer,} \quad \alpha=1,...,n$$



### QUBO Representation

We define two set of variables in this problem [1]:  
(1) $x_\alpha$ is a **discrete** variable which denotes the quantity of item $\alpha$ you decide to purchase.  
(2) $y_n$ is a **binary** variable which equals to 1 if the total cost of your purchase is equal to $n$.

The following Hamiltonian representing the problem.

$$H=A\left(1 - \sum_{n=1}^{W} y_{n}\right)^{2} + A \left(\sum_{n=1}^{W}n y_{n} - \sum_{\alpha=1}^{M} W_{\alpha} x_{\alpha}\right)^{2}  - B\sum_{\alpha=1}^{M}V_{\alpha}x_{\alpha}$$

The first term enforces the total cost is less than or equal to the budget $W$, because exactly one $w_{n}$ equals to 1. The second term enforces that the total cost is indeed the sum of the costs of each item to guarantee that this is a valid soltuion. The third term term is to maximize the profit. $H_3$ enforces the consistency that you can only purchase items from the suppliers that are chosen. The hyperparameters $A_1$, $A_1'$, $B_1$, $A_2$, $A_2'$, $B2$ and $C$ are Lagrange multipliers or penalty coefficients. We need to satisfy $A>B>0$ in order to get valid solutions. 

We expand the Hamiltonian $H$ to get the linear and quadratic terms in the QUBO representation.

$$H=\sum_{\alpha=1}^{M}Aw_{\alpha}^2x_{\alpha}^2 - \sum_{\alpha=1}^{M}B W_{\alpha} x_{\alpha} + \sum_{n=1}^{W}A (n^2-1)y_n + \sum_{n,m}^{W}2A\left(1+nm\right)y_ny_m + \sum_{\alpha,\beta}^{M}2A w_{\alpha}w_{\beta}x_{\alpha}x_{\beta}y_ny_m-\sum_{\alpha=1}^{M}\sum_{n=1}^{W}2Anw_{\alpha}y_nx_{\alpha}$$

The first two terms are linear terms while the rest are quadratic terms.

### Andrew Lucas Log trick: 
$y_n$ are auxiliary variables to implement the budget constaint $W$, so there are $W$ of such variables. Andrew Lucas introcued a trick to reduce the number of $y_n$ to $\log(W)$.

### Three DQMs:
We construct three DQMs below.
(1) `build_knapsack_dqm_fixedbound` construsts a DQM which assumes the bound is fixed for all items, with the log trick implemented.
(2) `build_knapsack_dqm_variablebound2` construsts a DQM which assumes the bound differs amont items, with the log trick implemented. 
(3) `build_knapsack_dqm_variablebound1` construsts a DQM which assumes the bound differs amont items, without using the log trick.

We first run DQM on a small dataset randomly generated to make sure the solution is the optimal one, then import a external grocery data to solve a real-world problem.

In [2]:
import os
os.chdir('..')
from utils.data import read_profit_optimization_data
from dimod import DiscreteQuadraticModel
from dimod import ExactSolver
import sys
from dwave.system import LeapHybridDQMSampler
from neal import SimulatedAnnealingSampler
from math import log2, floor
import dimod
import os
import numpy as np
import pandas as pd

## Construct DQM
### Fixed Bound

In [16]:
def build_knapsack_dqm_fixedbound(values, weights, weight_capacity, bound, verbose = False):
    """Construct DQM for the generalized knapsack problem
    Args:
        values (array-like):
            Array of values associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        bound(int):
            Maximum allowable pieces for each item
    Returns:
        Discrete quadratic model instance
        x: variable
    """
    bound += 1 # also take into account the value 0
    pieces = range(bound)
    
    # First guess the lagrange
    lagrange = max(values)*0.5
    if verbose:
        print('lagrange:',lagrange)

    # Number of objects
    x_size = len(values)

    # Lucas's algorithm introduces additional slack variables to
    # handle the inequality. M+1 binary slack variables are needed to
    # represent the sum using a set of powers of 2.
    M = floor(log2(weight_capacity))
    num_slack_variables = M + 1

    # Slack variable list for Lucas's algorithm. The last variable has
    # a special value because it terminates the sequence.
    y = [2**n for n in range(M)]
    y.append(weight_capacity + 1 - 2**M)
    
    ##@  Discrete Quadratic Model @##
    dqm = DiscreteQuadraticModel()
    x = []
    #@ Add variables @##
    for k in range(x_size):
        x.append(dqm.add_variable(bound, label='x' + str(k)))

    for k in range(num_slack_variables):
        dqm.add_variable(2, label='y' + str(k)) # either 0 or 1

    ##@ Hamiltonian xi-xi terms ##
    for k in range(x_size):
        dqm.set_linear('x' + str(k), lagrange * (weights[k]**2) * (np.array(pieces)**2) - values[k]*pieces)


    # # Hamiltonian xi-xj terms
    for i in range(x_size):
        for j in range(i + 1, x_size):
            biases_dict = {}
            for piece1 in pieces:
                for piece2 in pieces:
                    biases_dict[(piece1, piece2)]=(2 * lagrange * weights[i] * weights[j])*piece1*piece2

            dqm.set_quadratic('x' + str(i), 'x' + str(j), biases_dict)

    # Hamiltonian y-y terms
    for k in range(num_slack_variables):
        dqm.set_linear('y' + str(k), lagrange*np.array([0,1])* (y[k]**2))

    # Hamiltonian yi-yj terms 
    for i in range(num_slack_variables):
        for j in range(i + 1, num_slack_variables): 
            dqm.set_quadratic('y' + str(i), 'y' + str(j), {(1,1):2 * lagrange * y[i] * y[j]})

    # Hamiltonian x-y terms
    for i in range(x_size):
        for j in range(num_slack_variables):
            biases_dict = {}
            for piece1 in pieces:
                biases_dict[(piece1, 1)]=-2 * lagrange * weights[i] * y[j]*piece1

            dqm.set_quadratic('x' + str(i), 'y' + str(j), biases_dict) 
    
    return dqm, x

### Various Bounds

In [4]:
def build_knapsack_dqm_variablebound2(values, weights, weight_capacity, bound, verbose = False):
    """Construct DQM for the generalized knapsack problem
    Args:
        values (array-like):
            Array of values associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        bound(int):
            Maximum allowable pieces for each item
    Returns:
        Discrete quadratic model instance
    """
    bound = [b+1 for b in bound] # also take into account the value 0
#     pieces = range(bound)
    
    # First guess the lagrange
    lagrange = max(values)*0.5
    if verbose:
        print('lagrange:',lagrange)

    # Number of objects
    x_size = len(values)

    # Lucas's algorithm introduces additional slack variables to
    # handle the inequality. M+1 binary slack variables are needed to
    # represent the sum using a set of powers of 2.
    M = floor(log2(weight_capacity))
    num_slack_variables = M + 1

    # Slack variable list for Lucas's algorithm. The last variable has
    # a special value because it terminates the sequence.
    y = [2**n for n in range(M)]
    y.append(weight_capacity + 1 - 2**M)
    
    ##@  Discrete Quadratic Model @##
    dqm = DiscreteQuadraticModel()
    
    x = []
    #@ Add variables @##
    for k in range(x_size):
        x.append(dqm.add_variable(bound[k], label='x' + str(k)))

    for k in range(num_slack_variables):
        dqm.add_variable(2, label='y' + str(k)) # either 0 or 1

    ##@ Hamiltonian xi-xi terms ##
    for k in range(x_size):
        pieces = range(bound[k])
        dqm.set_linear('x' + str(k), lagrange * (weights[k]**2) * (np.array(pieces)**2) - values[k]*pieces)


    # # Hamiltonian xi-xj terms
    for i in range(x_size):
        for j in range(i + 1, x_size):
            biases_dict = {}
            for piece1 in range(bound[i]):
                for piece2 in range(bound[j]):
                    biases_dict[(piece1, piece2)]=(2 * lagrange * weights[i] * weights[j])*piece1*piece2

            dqm.set_quadratic('x' + str(i), 'x' + str(j), biases_dict)

    # Hamiltonian y-y terms
    for k in range(num_slack_variables):
        dqm.set_linear('y' + str(k), lagrange*np.array([0,1])* (y[k]**2))

    # Hamiltonian yi-yj terms 
    for i in range(num_slack_variables):
        for j in range(i + 1, num_slack_variables): 
            dqm.set_quadratic('y' + str(i), 'y' + str(j), {(1,1):2 * lagrange * y[i] * y[j]})

    # Hamiltonian x-y terms
    for i in range(x_size):
        for j in range(num_slack_variables):
            biases_dict = {}
            for piece1 in range(bound[i]):
                biases_dict[(piece1, 1)]=-2 * lagrange * weights[i] * y[j]*piece1

            dqm.set_quadratic('x' + str(i), 'y' + str(j), biases_dict) 
    
    return dqm,x

# Without the Andrew Lucas log trick
def build_knapsack_dqm_variablebound(values, weights, weight_capacity, bound, verbose = False):
    """Construct DQM for the generalized knapsack problem
    Args:
        values (array-like):
            Array of values associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        bound(array-like):
            Maximum allowable pieces for each item
    Returns:
        Discrete quadratic model instance
        x: varibles
    """
    bound = [b+1 for b in bound] # also take into account the value 0
    
    # Lagrange multipliers A>max(values)>0
    A1 = max(values)*8
    A2 = max(values)*2
    
    num_of_items = len(values)
    ##@  Discrete Quadratic Model @##
    dqm = DiscreteQuadraticModel()

    x = []
    #@ Add variables @##
    for k in range(num_of_items):
        x.append(dqm.add_variable(bound[k], label='x' + str(k))) # number of discrete values 

    for n in range(1,weight_capacity+1):
        dqm.add_variable(2, label='y' + str(n)) # either 0 or 1, 2 values possible

    ##@ Hamiltonian xi-xi terms ##
    for k in range(num_of_items):
        pieces = range(bound[k])
    #     dqm.set_linear('x' + str(k),  - values[k]*pieces)
        dqm.set_linear('x' + str(k), A2 * (weights[k]**2) * (np.array(pieces)**2) - values[k]*np.array(pieces))

    # Hamiltonian y-y terms
    for n in range(1,weight_capacity+1):
        dqm.set_linear('y' + str(n), np.array([0,1])* (n**2*A2-A1))

    # Hamiltonian yi-yj terms 
    for n in range(1,weight_capacity+1):
        for m in range(n + 1, weight_capacity+1): 
            dqm.set_quadratic('y' + str(n), 'y' + str(m), {(1,1):2 * A1 * (1+m*n)})

    # # Hamiltonian xi-xj terms
    for i in range(num_of_items):
        for j in range(i + 1, num_of_items):
            biases_dict = {}
            for piece1 in range(bound[i]):
                for piece2 in range(bound[j]):
                    biases_dict[(piece1, piece2)]=(2 * A2 * weights[i] * weights[j])*piece1*piece2
            dqm.set_quadratic('x' + str(i), 'x' + str(j), biases_dict)

    # Hamiltonian x-y terms
    for i in range(num_of_items):
        for n in range(1,weight_capacity+1):
            biases_dict = {}
            for piece1 in range(bound[i]):
                biases_dict[(piece1, 1)]=-2 * A2 * weights[i] * n* piece1

            dqm.set_quadratic('x' + str(i), 'y' + str(n), biases_dict) 
            
    return dqm, x

In [5]:
def solve_dqm(dqm, x, sampler = None, verbose = False):
    if sampler is None:
        sampler = LeapHybridDQMSampler()
    sampleset = sampler.sample_dqm(dqm)
    best_solution = sampleset.first.sample    
    best_solution = [best_solution[i] for i in x]
    print(best_solution)
    
    return best_solution

# Implementation

In [6]:
num_of_items = 12
values = list(np.random.randint(1,10, size=(num_of_items)))
weights = list(np.random.randint(1,10, size=(num_of_items)))
weight_capacity = np.random.randint(12, 40)
print('values:',values)
print('weights:',weights)
print('weight_capacity:',weight_capacity)

values: [2, 8, 2, 9, 7, 4, 2, 3, 4, 9, 9, 8]
weights: [4, 3, 8, 5, 6, 7, 1, 2, 6, 4, 6, 6]
weight_capacity: 18


### Solve Bounded Knapsack Problem 1: All Items have the Same Bound

In [7]:
fixed_bound = 3
print('fixed bound:',fixed_bound)

(dqm,x) = build_knapsack_dqm_fixedbound(values, weights, weight_capacity, fixed_bound)
best_solution = solve_dqm(dqm,x)

total_weights = sum([weights[i]*best_solution[i] for i in range(len(x))])
total_value = sum([values[i]*best_solution[i] for i in range(len(x))])
    
print('Total weight:',total_weights)
print('Total value:',total_value)

fixed bound: 3
[0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1]
Total weight: 18
Total value: 36


### Solve Bounded Knapsack Problem 2: Each Item has Different Bounds

In [7]:
# Define an array of bounds
variable_bounds = list(np.random.randint(2, 6,size=(num_of_items)))
# variable_bounds = [3 for i in range(num_of_items)]
print('variable bounds:',variable_bounds)

(dqm,x) = build_knapsack_dqm_variablebound(values, weights, weight_capacity, variable_bounds)
best_solution = solve_dqm(dqm,x)

total_weights = sum([weights[i]*best_solution[i] for i in range(len(x))])
total_value = sum([values[i]*best_solution[i] for i in range(len(x))])
    
print('Total weight:',total_weights)
print('Total value:',total_value)

variable bounds: [4, 4, 3, 5, 2, 5, 4, 3, 3, 4, 2, 3]
[0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 1, 0]
Total weight: 22
Total value: 41


### Solve Bounded Knapsack Problem 2: Each Item has Different Bounds (with Lucas log trick)

In [8]:
# Define an array of bounds
variable_bounds = list(np.random.randint(2, 6,size=(num_of_items)))
# variable_bounds = [3 for i in range(num_of_items)]
print('variable bounds:',variable_bounds)

(dqm,x) = build_knapsack_dqm_variablebound2(values, weights, weight_capacity, variable_bounds)
best_solution = solve_dqm(dqm,x)

total_weights = sum([weights[i]*best_solution[i] for i in range(len(x))])
total_value = sum([values[i]*best_solution[i] for i in range(len(x))])
    
print('Total weight:',total_weights)
print('Total value:',total_value)

variable bounds: [2, 5, 3, 2, 2, 5, 2, 2, 3, 4, 2, 3]
[0, 4, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0]
Total weight: 18
Total value: 45


# Grocery Data

In [12]:
profit, cost = read_profit_optimization_data(os.path.join(os.getcwd(),'data/small-cost-mock.csv'))
budget = np.mean(cost)*50*np.random.rand()

print('average cost: ', np.mean(cost))
print('average profit: ', np.mean(profit))
print('budget: ', budget)

average cost:  11.723847457000542
average profit:  23.447694914001083
budget:  454.74591076689495


We need to formulate everything as integers, so multiply by 100

In [13]:
profit_integers = np.array([int(p*100) for p in profit])
cost_integers = np.array([int(c*100) for c in cost])
budget_integer = int(budget*100)

### Solve Bounded Knapsack Problem 1: All Items have the Same Bound

In [14]:
fixed_bound = 50
print('fixed bound:',fixed_bound)

(dqm,x) = build_knapsack_dqm_fixedbound(profit_integers, cost_integers, budget_integer, fixed_bound)
best_solution = solve_dqm(dqm,x)

total_costs = sum([cost[index]*count for index, count in enumerate(best_solution)])
total_profit = sum([profit[index]*count for index, count in enumerate(best_solution)])
    
print('Total cost:',total_costs)
print('Total profit:',total_profit)

fixed bound: 50
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 40, 46]
Total cost: 403.2902700653666
Total profit: 806.5805401307332


### Solve Bounded Knapsack Problem 2: Each Item has Different Bounds

In [15]:
# Define an array of bounds using the log trick
variable_bounds = list(np.random.randint(10, 50,size=(len(profit))))
print('variable bounds:',variable_bounds)

(dqm,x) = build_knapsack_dqm_variablebound2(profit_integers, cost_integers, budget_integer, variable_bounds)
best_solution = solve_dqm(dqm,x)

total_costs = sum([cost[index]*count for index, count in enumerate(best_solution)])
total_profit = sum([profit[index]*count for index, count in enumerate(best_solution)])
    
print('Total cost:',total_costs)
print('Total profit:',total_profit)

variable bounds: [47, 47, 47, 32, 41, 17, 43, 15, 19, 34, 49, 12, 21, 11, 46, 17, 38, 36, 11, 43]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 3, 7, 9, 29]
Total cost: 415.4007797064619
Total profit: 830.8015594129236


## References
[1] Lucas, A., 2014. Ising formulations of many NP problems. Frontiers in physics, 2, p.5.